## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

/Users/hani/Documents/llm-zoomcamp/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"
vector = embedding_model.encode(user_question)
print("The first value of the embeded question: ", round(vector[0], 3))

The first value of the embeded question:  0.078


## Prepare the documents

In [3]:
# Load the documents with ids that we prepared in the module:

import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
ml_documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']
len(ml_documents)

375

In [5]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

## Q2. Creating the embeddings

In [6]:
# we will create an embedding for both question and answer fields.
# We want to put all of them into a single matrix `X`:

embeddings = []

for doc in ml_documents:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

import numpy as np
X = np.array(embeddings)
X.shape



(375, 768)

## Q3. Search

In [7]:
# let's compute the cosine similarity between the vector from Q1 
# and the matrix from Q2. 
scores = X.dot(vector)
print("Highest Score of Cosine Similarity Scores: ", round(scores.max(), 3))

Highest Score of Cosine Similarity Scores:  0.651


## Vector search

In [18]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine_result = search_engine.search(vector, num_results=5)
search_engine_result[0]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

## Q4. Hit-rate for our search engine
We will use the hitrate metric for evaluation.

In [17]:
# load the ground truth dataset
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [22]:
def hit_rate_cal(ground_truth, search_engine, num_results=5):
    cnt = 0
    for q in tqdm(ground_truth):
        query = q['question']
        doc_id_true = q['document']

        vector_query = embedding_model.encode(query)
        search_engine_result = search_engine.search(vector_query, num_results=num_results)

        if any(result['id'] == doc_id_true for result in search_engine_result):
            cnt += 1

    return cnt/len(ground_truth)

hit_rate = hit_rate_cal(ground_truth, search_engine, num_results=5)
print("Q4, Hit-rate for our search engine: ", round(hit_rate, 3))